## 背景

- **集体智慧**：为了创造新的想法，而将一群人的行为、偏好或思想组合在一起。
- **机器学习的局限**：机器学习算法受限于其在大量模式之上的归纳能力，且只能凭借已经见过的数据进行归纳。如果一个模式不同于算法先前所见的任何其他模式，那么它很有可能会被“误解”。
- **协作性过滤**：一个协作性过滤算法通常的做法是对一大群人进行搜索，并从中找出与我们品味相近的一群人。

## 搜集偏好

下面是一个涉及影评者及其对几部电影评分情况的字典：

In [15]:
from recommendations import critics
print(critics)
print("\nLisa Rose 对 Lady in the Water 的评分：",critics['Lisa Rose']['Lady in the Water'])
print("\nToby 评价过的电影及评分：", critics['Toby'])

{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0}, 'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}, 'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5, 'The Night Listener': 4.0}, 'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 2.5}, 'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0}, 'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5}, 'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman 

## 寻找相近的用户
搜集完人们的偏好数据之后，我们需要有一种方法来确定人们在品味方面的相似程度。为此，我们可以将每个人与所有其他人进行对比，并计算他们的**相似度评价值**。两套常用的计算相似度评价值的体系：**欧几里得距离**和**皮尔逊相关度**。

- **欧氏距离**：介绍略

In [24]:
# 通过欧氏距离计算两个人相似度评价的函数
from math import sqrt
def sim_distance(prefs, p1, p2):
    # 如果两人没有共同项，返回0
    sim = []
    for item in prefs[p1]:
        if item in prefs[p2]:
            sim.append(item)
            
    if len(sim) == 0:
        return 0
    
    # 计算欧氏距离
    distance = sqrt(sum([pow(prefs[p1][item]-prefs[p2][item], 2) for item in sim]))
    return 1/(1+distance) # 防止除数为0

In [25]:
sim_distance(critics, 'Lisa Rose', 'Gene Seymour')

0.29429805508554946

- **皮尔逊相关度**

该相关系数是判断两组数据与某一直线拟合程度的一种度量。它在数据不是很规范（normalized）的时候（比如，影评者对影片的评价总是相对于平均水平偏离很大时），会倾向于给出更好的结果。

In [26]:
# 通过皮尔逊相关系数计算相似度
def sim_pearson(prefs, p1, p2):
    # 如果两人没有共同项，返回1
    sim = []
    for item in prefs[p1]:
        if item in prefs[p2]:
            sim.append(item)
    
    n = len(sim)
    if n == 0:
        return 1
    
    # 求和
    sum1 = sum([prefs[p1][item] for item in sim])
    sum2 = sum([prefs[p2][item] for item in sim])
    
    # 求平方和
    sum1_sq = sum([pow(prefs[p1][item], 2) for item in sim])
    sum2_sq = sum([pow(prefs[p2][item], 2) for item in sim])
    
    # 求乘积之和
    p_sum = sum([prefs[p1][item]*prefs[p2][item] for item in sim])
    
    # 计算相关系数
    num = p_sum - (sum1*sum2/n)
    den = sqrt((sum1_sq-pow(sum1,2)/n)*(sum2_sq-pow(sum2,2)/n))
    
    if den==0:
        return 0
    r = num/den
    return r

In [28]:
sim_pearson(critics, 'Lisa Rose', 'Gene Seymour')

0.39605901719066977

## 为评论者打分
既然已经有了对两个人进行比较的函数，下面可以编写函数，根据指定的人对每个人进行打分，并找出最接近的匹配结果。

In [37]:
def topMatches(prefs, person, n=5, sim_func=sim_pearson):
    '''
    perfs: 偏好字典
    n: 返回最接近的n个结果
    sim_func: 评价函数
    '''
    # 返回（相似度，名字）的元组
    scores = [(sim_func(prefs, person, other), other) for other in prefs if other != person] 
    scores.sort()
    scores.reverse()
    return scores[:n]

In [38]:
topMatches(critics, 'Toby', n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]